# Preprocessing training set
This notebook preprocesses the training set from the raw data files, including:

- item, ort, pho: ../raw/Chang/6kdict
- pho to encoded pho: ../raw/Chang/mapping
- sem: ../raw/Chang/6ksem.mat
- wf: ../raw/ELP/elp_wf.csv

Outputs:
- training set (in a custom training/testing set format): ../train.pkl.gz
- human readable training set: ../train.csv
- tokenizers used for encoding orthography: ../ort_tokenizers.pkl.gz

In [1]:
import pickle, gzip
import pandas as pd
import tensorflow as tf
from scipy.io import loadmat
from data_wrangling import (
    trim_unused_slots, 
    get_duplicates, 
    ort_to_binary,
    pho_to_binary,
    gen_pkey,
    save_tokenizers
)

# Basic checking

Load training set raw data

In [2]:
train = pd.read_csv(
    "../raw/Chang/6kdict",
    sep=r"[\t| ]",
    header=None,
    names=["word", "ort", "pho", "sampling_weight"],
    na_filter=False,
)

train

/home/jal21012/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,word,ort,pho,sampling_weight
0,a,____a_________,___^______,1.000000
1,ace,____a_ce______,___es_____,0.077460
2,ache,____a_che_____,___ek_____,0.161245
3,ached,____a_ched____,___ekt____,0.232379
4,aches,____a_ches____,___eks____,0.141421
...,...,...,...,...
6224,zoo,___zoo________,__zu______,0.527257
6225,zoom,___zoom_______,__zum_____,0.126491
6226,zoomed,___zoomed_____,__zumd____,0.077460
6227,zooms,___zooms______,__zumz____,0.077460


Remove unused slots

In [3]:
train['ort'] = trim_unused_slots(train.ort)
train['pho'] = trim_unused_slots(train.pho)

We have these slots: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Removing unused slots: [0, 11, 12, 13]
We have these slots: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Removing unused slots: [8, 9]


AssertionError: 

Failed representation lenght safety check... 

In [ ]:
print(set(len(x) for x in train['pho']))  # We failed length check in phoneme column, some words has 11 slots in pho

Either 10 slots or 11 slots in pho

In [4]:
train.loc[train.pho.str.len() >= 11, ]

,word,ort,pho,sampling_weight
900,clutched,_clu_tched,_kl^tCt____,0.232379
1187,dang,__da_ng___,__d@ng_____,0.050000


For some reason, two words has len of 11 in PHO, the extra slot should be the last one. Manually trim it.

In [5]:
train.loc[train.pho == "_kl^tCt____",'pho'] = "_kl^tCt___"
train.loc[train.pho == "__d@ng_____",'pho'] = "__d@ng____"

Redo the slot trimming in pho

In [6]:
train['pho'] = trim_unused_slots(train.pho)

We have these slots: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Removing unused slots: [8, 9]


Check slot usage

In [7]:
slots_usage = {slot: set([x[slot] for x in train.pho]) for slot in range(len(train.pho[0]))}
print(slots_usage)

{0: {'_', 's'}, 1: {'T', 's', '_', 'b', 'd', 'S', 't', 'f', 'g', 'p', 'k'}, 2: {'s', 'd', 'h', 'S', 'p', 'T', 'l', 'w', 'm', 'n', '_', 'b', 'k', 'r', 'v', 'f', 'y', 'C', 'D', 't', 'z', 'g', 'J'}, 3: {'Y', 'i', 'a', 'u', 'U', 'e', 'O', 'W', 'o', '^', 'A', '@', 'I', 'E'}, 4: {'s', 'd', 'S', 'p', 'T', 'l', 'm', 'n', '_', 'b', 'Z', 'r', 'k', 'v', 'f', 'C', 'D', 't', 'z', 'g', 'J'}, 5: {'m', 'T', 's', 'n', '_', 'b', 'd', 'J', 'v', 'l', 'S', 't', 'f', 'z', 'g', 'p', 'C', 'k'}, 6: {'T', 's', '_', 'd', 't', 'z', 'J'}, 7: {'z', '_', 's', 't'}}


Check if the unique token can all be found in the mapping 

In [8]:
unique_token = {x for slot in slots_usage.values() for x in slot}
print(unique_token)
print(f"{len(unique_token)} unique token in the training set")

{'a', 's', 'U', 'd', 'h', 'S', 'A', 'p', 'E', 'Y', 'i', 'T', 'e', 'O', 'W', 'l', '@', 'w', 'I', 'm', 'u', 'n', '_', 'b', 'Z', 'r', 'v', 'o', 'f', 'y', 'C', 'J', 'D', 't', '^', 'z', 'g', 'k'}
38 unique token in the training set


In [9]:
pho_map = gen_pkey(key_file="../raw/Chang/mapping")

In [10]:
token_with_mapping = set(pho_map.keys())
print(token_with_mapping)
print(f"{len(token_with_mapping)} unique tokens in the mapping file")

{'a', 's', 'U', 'G', 'd', 'h', 'S', 'A', 'p', 'E', 'Y', 'i', 'T', 'e', 'O', '@', 'l', 'W', 'w', 'I', 'm', 'u', 'n', '_', 'b', 'Z', 'k', 'r', 'v', 'o', 'f', 'y', 'C', 'D', 't', '^', 'z', 'g', 'J'}
39 unique tokens in the mapping file


In [11]:
print("This token is not used in the data, but exists in the mapping file:")
set(pho_map.keys()).difference(unique_token)

This token is not used in the data, but exists in the mapping file:


{'G'}

G is not used in dataset

In [12]:
print("This token is used in the data, but not exists in the mapping file:")
unique_token.difference(pho_map.keys()) 

This token is used in the data, but not exists in the mapping file:


set()

All token can be found in pho_map. It should be safe to use this mapping dict along with the current training set 

# Get word frequency
Since Chang did not provide the raw word frequency, we need to obtain it from elsewhere. Perhaps we can use SUBTLWF obtained from ELP. 

In [13]:
df_wf = pd.read_csv('../raw/ELP/elp_wf.csv', index_col=None, na_filter='#', thousands=',')
df_wf.sample(10)

,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL,SUBTLWF
2005,1,gourd,5,185,5.220,0.470
1476,1,fair,4,45321,10.722,94.750
695,1,chin,4,4747,8.465,12.690
4543,1,snook,5,163,5.094,0.120
1949,1,gloats,6,57,4.043,#
1173,1,deal,4,83766,11.336,261.370
1798,1,fruits,6,4378,8.384,2.800
379,1,botch,5,243,5.493,0.310
4357,1,skimp,5,172,5.147,0.250
382,1,boughs,6,35,3.555,0.350


Parse

In [14]:
df_wf['SUBTLWF'] = df_wf['SUBTLWF'].str.replace(',', '')
df_wf['SUBTLWF'] = df_wf['SUBTLWF'].str.replace('#', '0')
df_wf['SUBTLWF'] = df_wf['SUBTLWF'].astype(float)

Merge

In [15]:
wf_for_merge = df_wf[['Word', 'SUBTLWF']].rename(columns={'Word': 'word', 'SUBTLWF': 'wf'})
train = train.merge(wf_for_merge, on='word', how='left')
train.pop('sampling_weight')

0       1.000000
1       0.077460
2       0.161245
3       0.232379
4       0.141421
          ...   
6224    0.527257
6225    0.126491
6226    0.077460
6227    0.077460
6228    0.252982
Name: sampling_weight, Length: 6229, dtype: float64

Also replace missing with 0 after merge

In [16]:
train['wf'] = train.wf.apply(lambda x: x if x >= 0 else 0)

# Check semantics (Wordnet)

In [17]:
wn_repr = loadmat('../raw/Chang/6ksem.mat')
wn_repr.keys()

dict_keys(['__header__', '__version__', '__globals__', 'numFeature', 'semantic_vector', 'maxFeature', 'tarWord'])

Safety check: Is 6ksem.mat is sorted in the same order as 6kdict?

In [18]:
wn_word_seq = [str(wn_repr['tarWord'][0, x][0]) for x in range(6229)]
assert wn_word_seq == train.word.tolist() 

Extract semantic vector from wordnet

In [19]:
sem = wn_repr['semantic_vector'].astype(float)
sem.shape

(6229, 2446)

Because word frequency is counted by each word (ort form), but there are some homographs in the dataset, we need to adjust the word frequency to avoid over-sampling in the homograph case. 

In [20]:
homographs = get_duplicates(train, 'ort')
print(f"There are {len(homographs)} homographs in the training set") 

There are 39 homographs in the training set


Show a homograph example.

In [21]:
train.loc[train.ort=='_shu_t____']

,word,ort,pho,wf
4496,shut,_shu_t____,__S^t___,263.82
4497,shut,_shu_t____,__S^t___,263.82


Now, proceed to adjusting word frequency to avoid over sampling from the duplicated words.

In [22]:
def adjust_wf(row):
    """Adjust word frequencies by the number of occurance."""
    if row.ort in homographs.keys():
        return(row.wf / homographs[row.ort])
    else:
        return(row.wf)
        
train['adjusted_wf'] = train.apply(adjust_wf, axis=1)

Print some examples to make sure it is working as expected.

In [23]:
train.loc[train.word.isin(['shut', 'man', 'swim'])]

,word,ort,pho,wf,adjusted_wf
3014,man,__ma_n____,__m@n___,1845.75,1845.75
4496,shut,_shu_t____,__S^t___,263.82,131.91
4497,shut,_shu_t____,__S^t___,263.82,131.91
5265,swim,_swi_m____,_swIm___,31.80,31.80


Save a human readable training set csv file for convienence reference.

In [24]:
train.to_csv('../train.csv')  # Checkpoint

# Encode training set

In [25]:
ort_np, ort_tokenizers = ort_to_binary(train.ort)

Token count: defaultdict(<class 'int'>, {'_': 6004, 'c': 4, 'p': 3, 's': 178, 't': 40})
Token count: defaultdict(<class 'int'>, {'_': 3772, 'b': 192, 'c': 393, 'h': 71, 'd': 80, 'f': 168, 'g': 174, 'k': 37, 'p': 191, 'q': 78, 'r': 5, 's': 735, 't': 236, 'w': 97})
Token count: defaultdict(<class 'int'>, {'_': 210, 'b': 305, 'l': 795, 'r': 1090, 'c': 305, 'h': 672, 'z': 24, 'd': 224, 'w': 305, 'f': 212, 'g': 174, 'n': 214, 'j': 97, 'k': 86, 'm': 289, 'p': 378, 's': 235, 'u': 83, 't': 391, 'v': 79, 'y': 61})
Token count: defaultdict(<class 'int'>, {'a': 1663, 'e': 1145, 'i': 1130, 'o': 1493, 'u': 737, 'y': 61})
Token count: defaultdict(<class 'int'>, {'_': 4374, 'i': 274, 'u': 241, 'w': 237, 'y': 89, 'a': 434, 'e': 353, 'h': 3, 'o': 224})
Token count: defaultdict(<class 'int'>, {'_': 239, 'c': 304, 'd': 347, 'f': 158, 'g': 270, 'l': 693, 'm': 377, 'r': 791, 's': 594, 'n': 915, 'p': 357, 't': 565, 'e': 35, 'x': 35, 'b': 168, 'z': 51, 'k': 187, 'u': 4, 'v': 126, 'q': 7, 'h': 4, 'w': 2})
Tok

Save orthographic tokenizer for later use.

In [26]:
save_tokenizers(ort_tokenizers, "../tokenizer/ort_tokenizers.pkl")

Package dataset

In [27]:
train_package = {
    "id": train.word.index.tolist(),
    "item": train.word.tolist(),
    "wf": train.adjusted_wf.tolist(),
    "ort": tf.constant(ort_np, dtype=tf.float32),
    "pho": tf.constant(pho_to_binary(train.pho, mapping=pho_map), dtype=tf.float32),
    "sem": tf.constant(sem, dtype=tf.float32),
    "graphem": train.ort.tolist(),  # 10 slots, vowel at slot 3 (0-indexed)
    "phoneme": train.pho.tolist(),  # 8 slots, vowel at slot 3 (0-indexed)
}


Verify dimensions are correct

In [28]:
print(len(train_package["id"]))
print(len(train_package["item"]))
print(len(train_package["wf"]))
print(train_package["ort"].shape)
print(train_package["pho"].shape)
print(train_package["sem"].shape)
print(len(train_package["graphem"]))
print(len(train_package["phoneme"]))

6229
6229
6229
(6229, 119)
(6229, 200)
(6229, 2446)
6229
6229


Export

In [29]:
with gzip.open("../train.pkl.gz", "wb") as f:
    pickle.dump(train_package, f)